* 요일: &we=[0, 2, 3, 4, 5, 6] <br>
* 전체, 홈, 원정: &ha, &ha=H or &ha=N <br>
* 구단(14-24): &ct=[1, 2, 3, 6, 7, 9, 11, 12, 13] <br>
기아: 2
삼성: 1
두산: 6
SSG: 9
롯데: 3
한화: 7
엔씨: 12
KT: 13
키움: 11

**컬럼명 해설**<br>
```python
df_kia_thu.columns

Index(['Year', 'G', 'GS', 'W', 'L', 'S', 'HD', 'IP', 'R', 'ER', 'RS', 'RS9',
       'TBF', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'IB', 'HP', 'SO', 'SF', 'NP',
       'WHIP', 'AVG', 'OBP', 'OPS', 'ERA'],
      dtype='object')
```

연도, 게임 수, 선발, 이긴 경기 수, 진 경기 수, 세이브, 홀드, 이닝 수, 득점, 자책점, 득점 지원, 9이닝당 득점지원, 상대한 타자 수, 상대팀 타수, 안타, 2루타, 3루타, 홈런, 4구, 고의 4구, 사구, 삼진, 희생플라이, 투구 수, 이닝달 출루 허용률, 피안타율, 피출루율, 피OPS, 평균자책점

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from re import escape
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

plt.rcParams['font.family'] = 'Malgun Gothic'

# 1. 기아타이거즈

## (1) 화요일

In [113]:
# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

df_kia_thu_list = []

where_list = ['H', 'N']

for where in where_list:
    # 페이지 열기
    url = "https://statiz.sporki.com/stats/?m=team&m2=pitching&m3=situation1&so=s_year&ob=&year=2024&sy=2014&ey=2024&te=5002&po=&lt=10100&reg=&pe=&ds=&de=&we=2&hr=&ha={}&ct=2&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=50&ph=&hs=&us=&na=&ls=&sf1=&sk1=&sv1=&sf2=&sk2=&sv2=".format(where)

    # 페이지 로드 대기
    time.sleep(3)

    # 페이지 소스 가져오기
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 해당 클래스의 div 태그 찾기
    div = soup.find('div', class_='table_type01 transverse_scroll cbox')

    if div:
        first_table = div.find('table')  # 첫 번째 table 태그만 가져옴

    #헤더 추출
    headers = [th.text.strip() for th in table.find_all("th")]

    #headers 원소 출력
    headers[:10], headers[10:20], headers[20:]

    #데이터프레임 컬럼을 만들기 위해 불필요한 헤더 삭제
    """rm = ['Rank', 'Season', 'Year▲', '비율', 'Year']    #삭제할 헤더명
    idx = [i for i, x in enumerate(headers) if x in rm] #삭제할 헤더 index

    columns = [x for i, x in enumerate(headers) if i not in idx]    #삭제 후 헤더
    columns[0] = 'Year' #정렬▲ → Year

    columns = np.array(columns) #Numpy 배열로 변경"""

    #테이블 데이터 추출
    rows = []
    for tr in table.find("tbody").find_all("tr"):
        cells = [td.text.strip() for td in tr.find_all("td")]
        rows.append(cells)

    rows = np.array(rows)

    """#'Rank', 'Season', 'Year▲' 값 제거
    rows = rows[:, 3:]"""

    #기아 화요일(잠실)
    df = pd.DataFrame(rows, columns=columns)

    df_kia_thu_list.append(df)

df_kia_thu = pd.concat(df_kia_thu_list)
df_kia_thu

ValueError: Shape of passed values is (8, 32), indices imply (8, 29)

---

In [13]:
# Chrome WebDriver 설정
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 페이지 열기
url = 'https://statiz.sporki.com/stats/?m=team&m2=pitching&m3=situation1&so=s_year&ob=&year=2024&sy=2014&ey=2024&te=5002&po=&lt=10100&reg=&pe=&ds=&de=&we=2&hr=&ha=H&ct=2&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=50&ph=&hs=&us=&na=&ls=&sf1=&sk1=&sv1=&sf2=&sk2=&sv2='

# 페이지 로드 대기
time.sleep(3)

# 페이지 소스 가져오기
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 해당 클래스의 div 태그 찾기
div = soup.find('div', class_='table_type01 transverse_scroll cbox')
table = div.find('table')

#헤더 추출
headers = [th.text.strip() for th in table.find_all("th")]

#headers 원소 출력
headers[:10], headers[10:20], headers[20:]

#데이터프레임 컬럼을 만들기 위해 불필요한 헤더 삭제
rm = ['Rank', 'Season', 'Year▲', '비율', 'Year']    #삭제할 헤더명
idx = [i for i, x in enumerate(headers) if x in rm] #삭제할 헤더 index

columns = [x for i, x in enumerate(headers) if i not in idx]    #삭제 후 헤더
columns[0] = 'Year' #정렬▲ → Year

columns = np.array(columns) #Numpy 배열로 변경

#테이블 데이터 추출
rows = []
for tr in table.find("tbody").find_all("tr"):
    cells = [td.text.strip() for td in tr.find_all("td")]
    rows.append(cells)

rows = np.array(rows)

#'Rank', 'Season', 'Year▲' 값 제거
rows = rows[:, 3:]

#기아 화요일(잠실)
df = pd.DataFrame(rows, columns=columns)

<div class="table_type01 transverse_scroll cbox">
<div style="position:sticky;top:0;left:0;width:100%;z-index:1;height:3rem;background-color:#E11956;font-weight:bold;color:#ffffff;line-height:3rem;font-size:1.4rem;text-indent:1rem;">리그 기록</div>
<table>
<thead>
<tr>
<th rowspan="2">Rank</th>
<th rowspan="2">Season</th>
<th class="sorting" ob="DESC" rowspan="2" so="s_year" style="width:5rem;max-width:5rem;">Year▲</th>
<th class="align_center sorting" ob="DESC" so="s_year">정렬▲</th>
<th class="sorting" flow="down" ob="DESC" rowspan="2" so="G" tooltip="Games, 출장">G</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="GS" tooltip="Games Started, 선발">GS</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="W" tooltip="Wins , 승">W</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="L" tooltip="Loses, 패">L</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="S" tooltip="Saves, 세이브">S</th><th class="sorting" flow="down" ob="DESC" rowspan="2" so="HD" tooltip="H

In [65]:
#헤더 추출
headers = [th.text.strip() for th in table.find_all("th")]

#headers 원소 출력
headers[:10], headers[10:20], headers[20:]

(['Rank', 'Season', 'Year▲', '정렬▲', 'G', 'GS', 'W', 'L', 'S', 'HD'],
 ['IP', 'R', 'ER', 'RS', 'RS9', 'TBF', 'AB', 'H', '2B', '3B'],
 ['HR',
  'BB',
  'IB',
  'HP',
  'SO',
  'SF',
  'NP',
  '비율',
  'ERA',
  'Year',
  'WHIP',
  'AVG',
  'OBP',
  'OPS'])

In [97]:
#데이터프레임 컬럼을 만들기 위해 불필요한 헤더 삭제
rm = ['Rank', 'Season', 'Year▲', '비율', 'Year']    #삭제할 헤더명
idx = [i for i, x in enumerate(headers) if x in rm] #삭제할 헤더 index

columns = [x for i, x in enumerate(headers) if i not in idx]    #삭제 후 헤더
columns[0] = 'Year' #정렬▲ → Year

columns = np.array(columns) #Numpy 배열로 변경
columns

array(['Year', 'G', 'GS', 'W', 'L', 'S', 'HD', 'IP', 'R', 'ER', 'RS',
       'RS9', 'TBF', 'AB', 'H', '2B', '3B', 'HR', 'BB', 'IB', 'HP', 'SO',
       'SF', 'NP', 'ERA', 'WHIP', 'AVG', 'OBP', 'OPS'], dtype='<U4')

In [98]:
columns.shape

(29,)

In [99]:
#테이블 데이터 추출
rows = []
for tr in table.find("tbody").find_all("tr"):
    cells = [td.text.strip() for td in tr.find_all("td")]
    rows.append(cells)

rows = np.array(rows)

#'Rank', 'Season', 'Year▲' 값 제거
rows = rows[:, 3:]
rows

array([['2015', '3', '1', '0', '1', '0', '0', '9.0', '4', '3', '3',
        '3.0', '36', '32', '6', '2', '0', '0', '3', '0', '1', '7', '0',
        '137', '1.00', '0.188', '0.278', '0.528', '3.00'],
       ['2016', '7', '1', '0', '0', '0', '1', '12.0', '6', '5', '6',
        '4.5', '54', '44', '9', '2', '0', '0', '9', '1', '0', '8', '0',
        '217', '1.50', '0.205', '0.340', '0.590', '3.75'],
       ['2017', '4', '1', '1', '0', '0', '0', '10.0', '3', '3', '4',
        '3.6', '40', '36', '6', '0', '0', '1', '4', '0', '0', '10', '0',
        '177', '1.00', '0.167', '0.250', '0.500', '2.70'],
       ['2019', '8', '2', '2', '0', '0', '0', '18.0', '5', '4', '24',
        '12.0', '72', '64', '10', '3', '0', '0', '6', '0', '2', '11',
        '0', '272', '0.89', '0.156', '0.250', '0.453', '2.00'],
       ['2020', '11', '2', '1', '1', '0', '0', '19.0', '13', '13', '10',
        '4.7', '89', '75', '22', '5', '0', '0', '11', '0', '1', '11',
        '0', '350', '1.74', '0.293', '0.391', '0.751'

In [100]:
rows.shape

(8, 29)

In [101]:
#기아 화요일(잠실)
df = pd.DataFrame(rows, columns=columns)

,Year,G,GS,W,L,S,HD,IP,R,ER,...,IB,HP,SO,SF,NP,ERA,WHIP,AVG,OBP,OPS
0,2015,3,1,0,1,0,0,9.0,4,3,...,0,1,7,0,137,1.00,0.188,0.278,0.528,3.00
1,2016,7,1,0,0,0,1,12.0,6,5,...,1,0,8,0,217,1.50,0.205,0.340,0.590,3.75
2,2017,4,1,1,0,0,0,10.0,3,3,...,0,0,10,0,177,1.00,0.167,0.250,0.500,2.70
3,2019,8,2,2,0,0,0,18.0,5,4,...,0,2,11,0,272,0.89,0.156,0.250,0.453,2.00
4,2020,11,2,1,1,0,0,19.0,13,13,...,0,1,11,0,350,1.74,0.293,0.391,0.751,6.16
5,2021,5,1,0,1,0,0,9.0,6,5,...,0,1,10,0,181,1.78,0.243,0.378,0.810,5.00
6,2022,13,2,0,2,0,0,18.0,15,14,...,0,1,15,1,308,1.50,0.282,0.350,0.801,7.00
7,2024,7,1,0,1,0,0,9.0,11,11,...,1,0,4,1,170,2.33,0.395,0.438,1.019,11.00


---

In [2]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

where_list = ['', 'H', 'N'] #전체, 홈, 원정
df_kia_thu_list = []

for where in where_list:
    # 페이지 열기
    url = 'https://statiz.sporki.com/stats/?m=team&m2=pitching&m3=situation1&so=s_year&ob=&year=2024&sy=2014&ey=2024&te=5002&po=&lt=10100&reg=&pe=&ds=&de=&we=2&hr=&ha={}&ct=2&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=50&ph=&hs=&us=&na=&ls=&sf1=&sk1=&sv1=&sf2=&sk2=&sv2='.format(where)

    # 페이지 열기
    driver.get(url)

    # 해당 클래스가 포함된 div가 나타날 때까지 대기 (최대 10초)
    try:
        div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "table_type01"))
        )
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        div_content = soup.find('div', class_='table_type01 transverse_scroll cbox')

        if div_content:
            first_table = div_content.find('table') #첫 번째 table 태그만 가져옴옴

        #헤더 추출
        headers = [th.text.strip() for th in first_table.find_all("th")]

        #데이터프레임 컬럼을 만들기 위해 불필요한 헤더 삭제
        rm = ['Rank', 'Season', 'Year▲', '비율', 'Year']    #삭제할 헤더명
        idx = [i for i, x in enumerate(headers) if x in rm] #삭제할 헤더 index

        columns = [x for i, x in enumerate(headers) if i not in idx]    #삭제 후 헤더
        columns[0] = 'Year' #정렬▲ → Year

        columns = np.array(columns)

        tr = div_content.find("tbody").find_all('tr')

        rows = []

        for tr in first_table.find('tbody').find_all('tr'):
            cells = [td.text.strip() for td in tr.find_all('td')]
            rows.append(cells)

        for i in range(len(rows)):
            rows[i] = rows[i][3:]

        df = pd.DataFrame(rows, columns=columns)
        df_kia_thu_list.append(df)

    except Exception as e:
        print("요소를 찾을 수 없음:", e)

    #print(url)

df_kia_thu = pd.concat(df_kia_thu_list) #홈, 원정 결합

#컬럼 순서 변경
col_list = list(df_kia_thu.columns)
col_list[-5:] = ['WHIP', 'AVG', 'OBP', 'OPS', 'ERA']
df_kia_thu.columns = col_list

df_kia_thu

,Year,G,GS,W,L,S,HD,IP,R,ER,...,IB,HP,SO,SF,NP,WHIP,AVG,OBP,OPS,ERA
0,2014,16,3,0,3,0,0,24.0,25,20,...,3,1,12,1,449,1.75,0.283,0.374,0.818,7.50
1,2015,14,4,3,1,1,2,38.0,13,10,...,1,2,35,1,564,1.00,0.206,0.268,0.547,2.37
2,2016,18,4,1,2,0,1,37.0,22,19,...,1,1,30,0,597,1.41,0.254,0.333,0.720,4.62
3,2017,14,3,1,2,0,0,28.0,12,12,...,1,2,25,1,498,1.36,0.248,0.333,0.743,3.86
4,2018,12,2,0,2,0,0,16.0,19,18,...,1,1,11,1,383,2.69,0.347,0.478,0.936,10.10
5,2019,12,3,3,0,0,0,27.0,9,8,...,0,2,15,0,414,1.15,0.233,0.295,0.577,2.67
6,2020,19,4,2,2,0,1,36.0,21,17,...,1,1,26,0,620,1.53,0.261,0.348,0.700,4.25
7,2021,8,2,0,2,0,0,17.0,13,12,...,0,1,12,0,316,1.88,0.324,0.407,0.914,6.35
8,2022,19,3,1,2,0,2,27.0,16,14,...,1,1,26,2,466,1.48,0.276,0.345,0.745,4.67
9,2023,5,1,1,0,1,1,9.0,3,3,...,1,0,2,0,128,1.44,0.286,0.342,0.771,3.00


In [3]:
df_kia_thu.shape

(30, 29)

In [4]:
df_kia_thu_total = df_kia_thu.iloc[:11]  #전체(홈+원정) 데이터프레임
df_kia_thu_home = df_kia_thu.iloc[11:19] #홈 데이터프레임
df_kia_thu_away = df_kia_thu.iloc[20:]   #원정 데이터프레임 